In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=vectorstore.as_retriever(),
)

chain.run("Victory Mansions를 묘사해주세요.")

'제가 알기로는 "승리 아파트"는 1984년의 소설에서 등장하는 건물로 제이지 오웰이 묘사한 곳입니다. 이 아파트는 런던에 위치하며, 낡고 초라한 상태로 묘사되어 있습니다. 건물은 무너져가는 상태이며, 엘리베이터가 자주 고장나는 등 불편한 환경이라고 합니다. 아파트는 냄새가 나고 혼잡하며, 주변 환경 또한 안전하지 않다고 묘사되어 있습니다.'